# CARTO Data Observatory. Access premium data.

This notebook shows how to use CARTOframes for discovering and downloading **premium** datasets from CARTO's [Data Observatory](https://carto.com/spatial-data-catalog/).

In particular, we will download touristic [POI's from Pitney Bowes](https://carto.com/spatial-data-catalog/browser/?category=points_of_interest&provider=pitney_bowes) within a specific bounding box.

The notebook is organized in the following sections:
 - [Check your subscriptions to premium datasets](#section1)
 - [Download a small sample of a dataset applying spatial filtering to explore it further](#section2)
 - [Download dataset filtering by column value and bounding box](#section3)
 - [Upload filtered dataset to your CARTO account](#section4)
 
 
**Note** for this notebook we are using the premium [dataset of Pitney Bowes POI's in Spain](https://carto.com/spatial-data-catalog/browser/dataset/pb_points_of_i_94bda91b/).

## Setup

### Import packages

In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)

from cartoframes import to_carto
from cartoframes.auth import set_default_credentials
from cartoframes.data.observatory import *
from cartoframes.viz import *
from shapely.geometry import box

### Set CARTO default credentials

In order to be able to use the Data Observatory via CARTOframes, you need to set your CARTO account credentials first.

Please, visit the [Authentication guide](https://carto.com/developers/cartoframes/guides/Authentication/) for further detail.

In [2]:
username = 'XXXXXXXXXX'
api_key  = 'XXXXXXXXXX'    # Master API key. Do not make this file public!

set_default_credentials(username=username, api_key=api_key)

**Note about credentials**

For security reasons, we recommend storing your credentials in an external file to prevent publishing them by accident when sharing your notebooks. You can get more information in the section _Setting your credentials_ of the [Authentication guide](https://carto.com/developers/cartoframes/guides/Authentication/).

## Download a premium dataset

When working with very large datasets, you might need to explore the dataset in detail to decide if you need the whole data or just part of it. In order to speed up your time to identifying the exact data, it might be very helpful to download just a small sample of your data, to later decide what you need. In this section, we will show how to identify toutistic POI's from a dataset we are already subscribed to.

<a id='section1'></a>
### Check your subscriptions to premium datasets

First, we check we're already subscribed to the dataset we want to use for our analysis. In this case, we would like to use [Pitney Bowes POI's in Spain](https://carto.com/spatial-data-catalog/browser/dataset/pb_points_of_i_94bda91b/). The dataset is `pb_points_of_i_94bda91b`.

You can subscribe to this premium [dataset](https://carto.com/spatial-data-catalog/browser/dataset/pb_points_of_i_94bda91b/) on your [CARTO dashboard](https://carto.com/help/working-with-data/subscribe_datasets_do/) or contacting CARTO.

In [5]:
Catalog().subscriptions().datasets.to_dataframe()

slug  \
0    ags_sociodemogr_f510a947   
1     pb_points_of_i_94bda91b   
2     sg_social_dist_667d8e8e   
3    mbi_consumer_pr_c4d89dd7   
4   u360_sociodemogr_28e93b81   
5    ags_consumer_sp_dbabddfb   
6     pb_consumer_po_62cddc04   
7     mc_geographic__7980c5c3   
8    spa_geosocial_s_d5dc42ae   
9     mc_consumer_jo_9e59134f   
10      vdf_footfall_c3eda566   

                                                 name  \
0   Sociodemographics - United States of America (...   
1     Points Of Interest - Spain (Latitude/Longitude)   
2   Social Distancing Metrics - United States of A...   
3            Consumer Profiles - Japan (Cho-Cho-Moku)   
4               Sociodemographics - Spain (Grid 100m)   
5   Consumer Spending - United States of America (...   
6   Points Of Interest - Consumer - United States ...   
7   Geographic Insights - United States of America...   
8   Geosocial Segments - United States of America ...   
9   Consumer Journey - United States of America (C...   
10                       Footfall - Spain (Grid 250m)   

                                          description         category_id  \
0   Census and ACS sociodemographic data estimated...        demographics   
1           Point of interest database per categories  points_of_interest   
2   Due to the COVID-19 pandemic, people are curre...             covid19   
3   The MB International Consumer Styles describe ...        demographics   
4   Curation of socio-demographic and socio-econom...        demographics   
5   The Consumer Expenditure database consists of ...        demographics   
6   Consumer Point of interest database per catego...  points_of_interest   
7   Geographic Insights validate, evaluate and ben...           financial   
8   By analysing feeds from Twitter, Instagram, Me...          behavioral   
9   Distribution of the total sales index and numb...           financial   
10  Count of unique visitors and total visits to a...      human_mobility   

   country_id               data_source_id   provider_id  \
0         usa            sociodemographics           ags   
1         esp           points_of_interest  pitney_bowes   
2         usa            social_distancing     safegraph   
3         jpn            consumer_profiles           mbi   
4         esp            sociodemographics      unica360   
5         usa            consumer_spending           ags   
6         usa  consumer_points_of_interest  pitney_bowes   
7         usa          geographic_insights    mastercard   
8         usa           geosocial_segments    spatial_ai   
9         usa             consumer_journey    mastercard   
10        esp                     footfall      vodafone   

                                       geography_name  \
0   Census Block Group - United States of America ...   
1                          Latitude/Longitude - Spain   
2   Census Block Group - United States of America ...   
3                                Cho-Cho-Moku - Japan   
4                                           Grid 100m   
5       Census Block Group - United States of America   
6       Latitude/Longitude - United States of America   
7   Census Block Group - United States of America ...   
8   Census Block Group - United States of America ...   
9   5-digit Zip Code Tabulation Area - United Stat...   
10                                  Grid 250m - Spain   

                                geography_description temporal_aggregation  \
0   Shoreline clipped TIGER/Line boundaries. More ...               yearly   
1                      Location of Points of Interest              monthly   
2   Shoreline clipped TIGER/Line boundaries. More ...                daily   
3   MBI Digital Boundaries for Japan at Cho-Cho-Mo...               yearly   
4                                       100x100m grid               yearly   
5                                                None               yearly   
6                      Location of Points of Interest             

In [4]:
pois_ds = Dataset.get('pb_points_of_i_94bda91b')

In [5]:
pois_ds.head()

HTTP ISO3                      NAME  SIC1  SIC2  SIC8  \
0  None  ESP                  EL BOLEO  0119  None  None   
1  None  ESP               PICO ANAYET  0119  None  None   
2  None  ESP                   CABAÑAS  0119  None  None   
3  None  ESP          PIC DE CASAMANYA  0119  None  None   
4  None  ESP             HOYO POLLALES  0119  None  None   
5  None  ESP             PICO TESORERO  0119  None  None   
6  None  ESP             PICA D'ESTATS  0119  None  None   
7  None  ESP                 PEÑARROYA  0119  None  None   
8  None  ESP  PUERTO DE PIEDRASLUENGAS  0119  None  None   
9  None  ESP           PICO DE LA CRUZ  0119  None  None   

                         CLASS                      EMAIL  \
0  FRUIT AND VEGETABLE MARKETS                       None   
1  FRUIT AND VEGETABLE MARKETS                       None   
2  FRUIT AND VEGETABLE MARKETS                       None   
3  FRUIT AND VEGETABLE MARKETS                       None   
4  FRUIT AND VEGETABLE MARKETS                       None   
5  FRUIT AND VEGETABLE MARKETS                       None   
6  FRUIT AND VEGETABLE MARKETS                       None   
7  FRUIT AND VEGETABLE MARKETS                       None   
8  FRUIT AND VEGETABLE MARKETS                       None   
9  FRUIT AND VEGETABLE MARKETS  ORDERS@LEWISSTAGNETTO.COM   

                             GROUP       PB_ID STABB  \
0  AGRICULTURAL PRODUCTION - CROPS  1171969022  None   
1  AGRICULTURAL PRODUCTION - CROPS  1198815451  None   
2  AGRICULTURAL PRODUCTION - CROPS  1172381728  None   
3  AGRICULTURAL PRODUCTION - CROPS  1394044479    GA   
4  AGRICULTURAL PRODUCTION - CROPS  1170858166    GA   
5  AGRICULTURAL PRODUCTION - CROPS  1394844928    GA   
6  AGRICULTURAL PRODUCTION - CROPS  1512254206    GA   
7  AGRICULTURAL PRODUCTION - CROPS  1510135375    GA   
8  AGRICULTURAL PRODUCTION - CROPS  1510749875    GA   
9  AGRICULTURAL PRODUCTION - CROPS  1507940952    GA   

                                               geoid        FAXNUM    MICODE  \
0  1171802457#-0.45805998654203861#39.47692747881...  943 65 28 25  10050119   
1                       1277532364#-5.41708#36.28679          None  10050119   
2   1173391073#1.7747074219121459#41.496068459223544          None  10050119   
3  1216381210#-4.1709914630209965#39.033559657297928          None  10050119   
4                        1318093445#0.89972#41.76093          None  10050119   
5    1172334602#-8.63541830839736#42.451687326575147          None  10050119   
6  1484024544#-0.43214432455418661#38.39538502859...          None  10050119   
7  1496527359#-8.7064276330132255#42.222296870656052          None  10050119   
8  1171592892#-6.0290784414220031#37.384497893124824          None  10050119   
9                        1171103654#-1.1288#37.98729          None  10050119   

        TEL_NUM     do_date   LATITUDE OPEN_24H POSTCODE  \
0          None  2020-09-01  43.024632     None    28017   
1          None  2020-09-01  42.774490     None    08029   
2          None  2020-09-01  37.810937     None    04140   
3          None  2020-09-01  42.584607     None    11405   
4          None  2020-09-01  40.887435     None    08830   
5          None  2020-09-01  43.182406     None    03003   
6          None  2020-09-01  42.679283     None    28915   
7          None  2020-09-01  40.377880     None    20590   
8  944 03 08 21  2020-09-01  43.046132     None    38470   
9          None  2020-09-01  28.744178     None    09400   

                                     do_label             AREANAME1  \
0                          EL TOBOSO - ALGABA        ISLAS CANARIAS   
1                            MUNTANER-LONDRES       CASTILLA Y LEÓN   
2                     ARAQUIL - AV. OSA MAYOR      REGIÓN DE MURCIA   
3            CTRA.A1-CRUCE COTOS DE MONTERREY      REGIÓN DE MURCIA   
4                      ESFINGE - SAN FAUSTINO               GALICIA   
5          GRAN AVENIDA - Cº VIEJO VILLAVERDE              CATALUÑA   
6      ARROYO 

Let's make sure the dataset covers our area of interest.

In [6]:
pois_ds.geom_coverage()

<a id='section2'></a>
### Download a small sample of a dataset applying spatial filtering to explore it further

We're only interested in tourism related POI's in Spain. Since we don't know exactly which variable to use in order to filter tourism POI's, we'll first download a small sample of the dataset to explore it. We'll filter by a bounding box covering Madrid downtown to make sure we have a good variety of POI's.

We can use SQL queries to specify the bounding box or polygon we are interested in.
- If you'd like to filter by bounding box, you need to use the SQL geography function `ST_IntersectsBox`.
- If you'd like to filter by polygon, you need to use the SQL geography function `ST_Intersects`.

In order to get the bounding box of interest we'll use [bboxfinder](https://bboxfinder.com).

In [7]:
sql_query = "SELECT * except(do_label) FROM $dataset$ WHERE ST_IntersectsBox(geom, -3.707628,40.415947,-3.700891,40.421403)"
sample_df = pois_ds.to_dataframe(sql_query=sql_query)

#To keep only most updated POI's (based on the do_date)
sample_df = sample_df.sort_values(['NAME', 'do_date']).groupby('NAME').first().reset_index()

sample_df.head()

/Users/malvarez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  


NAME  \
0                                     100 MONTADITOS   
1                               1005 DISTRIBUCION SL   
2                       15K ANGELS AND INVESTORS SL.   
3                        1610 PLAZA DEL CARMEN 5 SL.   
4  1ST WANDA SERVICE SERVICIOS INTEGRADOS DE CONT...   

                                               geoid     do_date BRANDNAME  \
0                         2173220473#-3.70582#40.416  2019-12-01       NaN   
1  1277167953#-3.7055463461631111#40.420295192309013  2019-12-01       NaN   
2                       1369422585#-3.70587#40.42048  2019-12-01       NaN   
3                        1289731188#-3.7028#40.41892  2019-12-01       NaN   
4                       1277282874#-3.70641#40.41956  2019-12-01       NaN   

        PB_ID      TRADE_NAME FRANCHISE_NAME ISO3 AREANAME4 AREANAME3  \
0  2173220473  100 MONTADITOS            NaN  ESP       NaN    MADRID   
1  1277167953             NaN            NaN  ESP       NaN    MADRID   
2  1369422585             NaN            NaN  ESP    MADRID    MADRID   
3  1289731188             NaN            NaN  ESP    MADRID    MADRID   
4  1277282874             NaN            NaN  ESP    MADRID    MADRID   

  AREANAME2            AREANAME1 STABB  POSTCODE  \
0    MADRID  COMUNIDAD DE MADRID    MD   28012.0   
1    MADRID               MADRID    MD   28013.0   
2    MADRID               MADRID    MD   28013.0   
3    MADRID               MADRID    MD   28013.0   
4    MADRID               MADRID    MD   28013.0   

                                   FORMATTEDADDRESS  \
0            CALLE POSTAS 12, 28012, MADRID, MADRID   
1             CALLE GRAN VIA, 28013, MADRID, MADRID   
2          CALLE GRAN VIA 46, 28013, MADRID, MADRID   
3             PLAZA CARMEN 5, 28013, MADRID, MADRID   
4  CALLE PRECIADOS, 29 - 5 A, 28013, MADRID, MADRID   

             MAINADDRESSLINE        ADDRESSLASTLINE  LONGITUDE   LATITUDE  \
0            CALLE POSTAS 12  28012, MADRID, MADRID  -3.705820  40.416000   
1             CALLE GRAN VIA  28013, MADRID, MADRID  -3.705546  40.420295   
2          CALLE GRAN VIA 46  28013, MADRID, MADRID  -3.705870  40.420480   
3             PLAZA CARMEN 5  28013, MADRID, MADRID  -3.702800  40.418920   
4  CALLE PRECIADOS, 29 - 5 A  28013, MADRID, MADRID  -3.706410  40.419560   

    GEORESULT CONFIDENCE_CODE  COUNTRY_ACCESS_CODE       TEL_NUM  \
0  S8HPNTSCZA            HIGH                  NaN  915 23 11 40   
1  S4-PNTSCZA          MEDIUM                 34.0  915 22 16 12   
2  S8HPNTSCZA            HIGH                 34.0           NaN   
3  S8HPNTSCZA            HIGH                 34.0           NaN   
4  S8HPNTSCZA            HIGH                 34.0           NaN   

         FAXNUM                         EMAIL                     HTTP  \
0  913 51 90 03  ATTCLIENTE@GRUPORESTALIA.COM  SPAIN.100MONTADITOS.COM   
1           NaN                           NaN                      NaN   
2           NaN                           NaN        WWW.15KANGELS.COM   
3           NaN                           NaN                      NaN   
4           NaN                           NaN                      NaN   

   OPEN_24H                              BUSINESS_LINE    SIC1  SIC2  \
0       NaN                                        NaN     NaN   NaN   
1       NaN         METALS SERVICE CENTERS AND OFFICES  5051.0   NaN   
2       NaN  SECURITY AND COMMODITY SERVICES, NEC, NSK  6289.0   NaN   
3       NaN                              EATING PLACES  5812.0   NaN   
4       NaN                BUSINESS SERVICES, NEC, NSK  7389.0   NaN   

         SIC8                    SIC8_DESCRIPTION  ALT_INDUSTRY_CODE  \
0         NaN                   TAPAS RESTAURANTS                NaN   
1  50510000.0  METALS SERVICE CENTERS AND OFFICES              350.0   
2  62890000.0      SECURITY AND COMMODITY SERVICE              350.0   
3  58120000.0                       EATING PLACES              350.0   
4  73890900.0                  FINANCIAL SERVICES

After some exploration of the DataFrame, we find out `TRADE_DIVISION` is our variable. There is a category called `DIVISION L. - TOURISM`.

In [8]:
sample_df['TRADE_DIVISION'].value_counts()

DIVISION I. - SERVICES                               1701
DIVISION G. - RETAIL TRADE                           1364
DIVISION H. - FINANCE, INSURANCE, AND REAL ESTATE     582
DIVISION E. - TRANSPORTATION AND PUBLIC UTILITIES     514
DIVISION F. - WHOLESALE TRADE                         201
DIVISION D. - MANUFACTURING                           188
DIVISION C. - CONSTRUCTION                             65
DIVISION J. - PUBLIC ADMINISTRATION                    24
DIVISION L. - TOURISM                                  14
DIVISION A. - AGRICULTURE, FORESTRY, AND FISHING       10
DIVISION M. - SPORTS                                    8
Name: TRADE_DIVISION, dtype: int64

#### Visualize the data sample

In [9]:
Map(Layer(sample_df, 
          geom_col='geom',
          style=color_category_style('TRADE_DIVISION', size=3, stroke_width=0.2), 
          encode_data=False))

<a id='section3'></a>
### Download dataset filtering by column value and bounding box

Once we know we can use column `TRADE_DIVISION` to filter tourism POIs, we'll add it to our query with a larger bounding box covering our area of interest.

In [10]:
sql_query = """
  SELECT * except(do_label) FROM $dataset$ 
  WHERE TRADE_DIVISION = 'DIVISION L. - TOURISM' 
  AND ST_IntersectsBox(geom, -3.716398,40.407437,-3.690477,40.425277)
"""
tourism_pois = pois_ds.to_dataframe(sql_query=sql_query)

#To keep only most updated POIs (based on the do_date)
tourism_pois = tourism_pois.sort_values(['NAME', 'do_date']).groupby('NAME').first().reset_index()

tourism_pois.head()

NAME  \
0                            ARCO DE CUCHILLEROS   
1                               ATENEO DE MADRID   
2                                BANCO DE ESPAÑA   
3  BASÍLICA DE NUESTRO PADRE JESÚS DE MEDINACELI   
4                  BIBLIOTECA NACIONAL DE ESPAÑA   

                              geoid     do_date  BRANDNAME       PB_ID  \
0   2033946578#-3.708101#40.4147983  2019-12-01        NaN  2033946578   
1  2033838561#-3.6982188#40.4150308  2019-12-01        NaN  2033838561   
2  2022507776#-3.6939777#40.4189737  2019-12-01        NaN  2022507776   
3  2033893910#-3.6957089#40.4141956  2019-12-01        NaN  2033893910   
4  2033804926#-3.6906236#40.4238526  2019-12-01        NaN  2033804926   

   TRADE_NAME  FRANCHISE_NAME ISO3  AREANAME4 AREANAME3 AREANAME2  \
0         NaN             NaN  ESP        NaN    MADRID    MADRID   
1         NaN             NaN  ESP        NaN    MADRID    MADRID   
2         NaN             NaN  ESP        NaN    MADRID    MADRID   
3         NaN             NaN  ESP        NaN    MADRID    MADRID   
4         NaN             NaN  ESP        NaN    MADRID    MADRID   

             AREANAME1 STABB  POSTCODE  \
0  COMUNIDAD DE MADRID    MD   28012.0   
1  COMUNIDAD DE MADRID    MD   28014.0   
2  COMUNIDAD DE MADRID    MD       NaN   
3  COMUNIDAD DE MADRID    MD   28014.0   
4  COMUNIDAD DE MADRID    MD   28001.0   

                                  FORMATTEDADDRESS           MAINADDRESSLINE  \
0  CALLE DE LOS CUCHILLEROS, 28012, MADRID, MADRID  CALLE DE LOS CUCHILLEROS   
1           CALLE DEL PRADO, 28014, MADRID, MADRID           CALLE DEL PRADO   
2                                   MADRID, MADRID                       NaN   
3            PLAZA DE JESÚS, 28014, MADRID, MADRID            PLAZA DE JESÚS   
4        PASEO DE RECOLETOS, 28001, MADRID, MADRID        PASEO DE RECOLETOS   

         ADDRESSLASTLINE  LONGITUDE   LATITUDE GEORESULT CONFIDENCE_CODE  \
0  28012, MADRID, MADRID  -3.708101  40.414798       T20             LOW   
1  28014, MADRID, MADRID  -3.698219  40.415031       T20             LOW   
2         MADRID, MADRID  -3.693978  40.418974        T1            HIGH   
3  28014, MADRID, MADRID  -3.695709  40.414196       T20             LOW   
4  28001, MADRID, MADRID  -3.690624  40.423853       T20             LOW   

   COUNTRY_ACCESS_CODE TEL_NUM FAXNUM  EMAIL HTTP  OPEN_24H  BUSINESS_LINE  \
0                  NaN     NaN    NaN    NaN  NaN       NaN            NaN   
1                  NaN     NaN    NaN    NaN  NaN       NaN            NaN   
2                  NaN     NaN    NaN    NaN  NaN       NaN            NaN   
3                  NaN     NaN    NaN    NaN  NaN       NaN            NaN   
4                  NaN     NaN    NaN    NaN  NaN       NaN            NaN   

   SIC1  SIC2  SIC8  SIC8_DESCRIPTION  ALT_INDUSTRY_CODE    MICODE  \
0   NaN   NaN   NaN              ARCH                NaN  10110112   
1   NaN   NaN   NaN  TOURIST BUILDING                NaN  10110200   
2   NaN   NaN   NaN  TOURIST BUILDING                NaN  10110200   
3   NaN   NaN   NaN  TOURIST BUILDING                NaN  10110200   
4   NaN   NaN   NaN  TOURIST BUILDING                NaN  10110200   

          TRADE_DIVISION    GROUP                         CLASS  \
0  DIVISION L. - TOURISM  TOURISM  IMPORTANT TOURIST ATTRACTION   
1  DIVISION L. - TOURISM  TOURISM  IMPORTANT TOURIST ATTRACTION   
2  DIVISION L. - TOURISM  TOURISM  IMPORTANT TOURIST ATTRACTION   
3  DIVISION L. - TOURISM  TOURISM  IMPORTANT TOURIST ATTRACTION   
4  DIVISION L. - TOURISM  TOURISM  IMPORTANT TOURIST ATTRACTION   

          SUB_CLASS  EMPLOYEE_HERE  EMPLOYEE_COUNT  YEAR_START  \
0              ARCH            NaN             NaN         NaN   
1  TOURIST BUILDING            NaN             NaN         NaN   
2  TOURIST BUILDING            NaN             NaN         NaN   
3  TOURIST BUILDING            NaN             NaN         NaN   
4  TOURIST BUILDING            NaN             NaN       

We can classify the tourism POI's using the variable `SUB_CLASS`.

In [11]:
tourism_pois['SUB_CLASS'].value_counts()

TOURIST BUILDING                             66
MONUMENT                                     20
IMPORTANT TOURIST ATTRACTION -UNSPECIFIED    16
SCENIC, PANORAMIC VIEW                        4
TOWER                                         2
STATUE                                        2
IMPORTANT TOURIST ATTRACTION                  2
ARCH                                          1
MEMORIAL                                      1
Name: SUB_CLASS, dtype: int64

In [12]:
Map(Layer(tourism_pois, 
          geom_col='geom',
          style=color_category_style('SUB_CLASS', stroke_width=0.2)))

<a id='section4'></a>
### Upload filtered dataset to your CARTO account

Finally, we will save this filtered dataset on our CARTO account so that we can use it in the future. We can do this using the CARTOframes function `to_carto`.

In [13]:
to_carto(tourism_pois, 'demo_tourism_pois_madrid', geom_col='geom')

Success! Data uploaded to table "demo_tourism_pois_madrid" correctly


'demo_tourism_pois_madrid'